# Graph Visualization with ipycytoscape
This notebook demonstrates the visualization of subgraphs from the [Neo4j](https://neo4j.com/) Graph Database. It uses the [py2neo](https://py2neo.org/) library to access a Neo4j database instance.

The examples in this notebook access the [COVID-19-Net](https://github.com/covid-19-net/covid-19-community) Knowledge Graph.

Author: Peter W. Rose (pwrose@ucsd.edu)

In [2]:
!pip install ipycytoscape

You should consider upgrading via the 'C:\Users\Dimitar\Anaconda3\envs\jupyterbook_env\python.exe -m pip install --upgrade pip' command.



  Using cached networkx-2.5-py3-none-any.whl (1.6 MB)


In [4]:
!pip install py2neo==2021.0.1

  Using cached six-1.15.0-py2.py3-none-any.whl (10 kB)
  Created wheel for neotime: filename=neotime-1.7.4-py3-none-any.whl size=20541 sha256=694a6f5e9b69da01f5bf38f088134ac8f2a5b71a0c0fba3074d354ae10bf3bfe
  Stored in directory: c:\users\dimitar\appdata\local\pip\cache\wheels\1b\53\64\de72baae858a94a3ed2c6b1117edf988b2b8d57b798097e5b3
Successfully built neotime
  Attempting uninstall: six
    Found existing installation: six 1.14.0
    Uninstalling six-1.14.0:
      Successfully uninstalled six-1.14.0
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 3.0.3
    Uninstalling prompt-toolkit-3.0.3:
      Successfully uninstalled prompt-toolkit-3.0.3


ERROR: nbinteract 0.2.6 has requirement nbformat<5,>=4.4.0, but you'll have nbformat 5.1.2 which is incompatible.
You should consider upgrading via the 'C:\Users\Dimitar\Anaconda3\envs\jupyterbook_env\python.exe -m pip install --upgrade pip' command.


In [5]:
import random
import ipycytoscape
from py2neo import Graph

#### Node and edge styles

In [6]:
node_centered = {'selector': 'node',
                 'style': {'font-size': '10',
                           'label': 'data(name)',
                           'height': '60',
                           'width': '80',
                           'text-max-width': '80',
                           'text-wrap': 'wrap',
                           'text-valign': 'center',
                           'background-color': 'blue',
                           'background-opacity': 0.6}
             }

In [7]:
edge_directed = {'selector': 'edge',
                 'style':  {'line-color': '#9dbaea',
                            'target-arrow-shape': 'triangle',
                            'target-arrow-color': '#9dbaea',
                            'curve-style': 'bezier'}
                }

In [8]:
edge_directed_named = {'selector': 'edge',
                       'style':  {'font-size': '8',
                                  'label': 'data(name)',
                                  'line-color': '#9dbaea',
                                  'text-rotation': 'autorotate',
                                  'target-arrow-shape': 'triangle',
                                  'target-arrow-color': '#9dbaea',
                                  'curve-style': 'bezier'}
                }

In [9]:
edge_undirected = {'selector': 'edge',
                   'style':  {'line-color': '#9dbaea'}
                  }

#### Node colors
Change seed to select a different color palette.

In [10]:
def random_color_palette(n_colors, seed=6):
    """ 
    Creates a random color palette of n_colors 
    See https://stackoverflow.com/questions/28999287/generate-random-colors-rgb
    
    """
    random.seed(seed)
    return ['#'+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(n_colors)]

### Connect to a Neo4j Database
Here we use the [COVID-19-Net](https://github.com/covid-19-net/covid-19-community) Knowledge Graph to demonstrate how to run a Neo4j Cypher query and pass the resulting subgraph into Cytoscape.

In [14]:
graph = Graph("bolt://132.249.238.185:7687", user="reader", password="demo")

### Example 1: Find all cities with the name "San Francisco"
This query demonstrates the geographic hierachy in COVID-19-Net.

In [15]:
query1 = """
MATCH p=(:City{name:'Skopje'})-[:IN*]->(:World) RETURN p
"""
subgraph1 = graph.run(query1).to_subgraph()

#### Add Neo4j subgraph to Cytoscape Widget

In [16]:
widget1 = ipycytoscape.CytoscapeWidget()
widget1.graph.add_graph_from_neo4j(subgraph1)

AttributeError: 'Graph' object has no attribute 'add_graph_from_neo4j'

#### Set node and edge styles

In [ ]:
style1 = [node_centered, edge_directed]

#### Set node specific colors based on node labels

In [ ]:
labels1 = list(subgraph1.labels())
print('Node labels:', labels1)

In [106]:
colors1 = random_color_palette(len(labels1))

In [107]:
for label, color in zip(labels1, colors1):
    style1.append({'selector': 'node[label = "' + label + '"]', 'style': {'background-color': color}})

In [108]:
widget1.set_style(style1)

In [109]:
widget1.set_layout(name='dagre', padding=0)

When a Neo4j subgraph is added to a Cytoscape graph, a tooltip attribute is generated that contains all Neo4j node properties.

In [110]:
widget1.set_tooltip_source('tooltip')

Click on a node to show the tooltip

In [111]:
widget1

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'padding': 0}, cytoscape_style=[{'selector': 'node', 'style…

### Example 2: Find all proteins that interact with the SARS-CoV-2 Spike protein
Here we run an undirected query (no "->" arrow) since the direction of interaction is arbitrary.

In [112]:
query2 = """
MATCH p=(:Protein{name: 'Spike glycoprotein', taxonomyId: 'taxonomy:2697049'})-[:INTERACTS_WITH]-(:Protein) RETURN p
"""
subgraph2 = graph.run(query2).to_subgraph()

In [113]:
widget2 = ipycytoscape.CytoscapeWidget()
widget2.graph.add_graph_from_neo4j(subgraph2)

In [114]:
style2 = [node_centered, edge_undirected]

In [115]:
labels2 = list(subgraph2.labels())
print('Node labels:', labels2)

Node labels: ['Protein']


In [116]:
colors2 = random_color_palette(len(labels2))

In [117]:
for label, color in zip(labels2, colors2):
    style2.append({'selector': 'node[label = "' + label + '"]', 'style': {'background-color': color}})

In [118]:
widget2.set_style(style2)

In [119]:
widget2.set_layout(name='concentric', padding=0)

In [120]:
widget2.set_tooltip_source('tooltip')

Click on a node to show the tooltip

In [121]:
widget2

CytoscapeWidget(cytoscape_layout={'name': 'concentric', 'padding': 0}, cytoscape_style=[{'selector': 'node', '…

### Example 3: Explore the Data Sources used to create the COVID-19-Net Knowledge Graph

In [122]:
query3 = """
MATCH p=(:MetaNode)-[:ETL_FROM]->(:DataSource) RETURN p  // ETL_FROM: Extracted, transformed, and loaded FROM
"""
subgraph3 = graph.run(query3).to_subgraph()

In [123]:
widget3 = ipycytoscape.CytoscapeWidget()
widget3.graph.add_graph_from_neo4j(subgraph3)

In [124]:
style3 = [node_centered, edge_directed]

In [125]:
labels3 = list(subgraph3.labels())
print('Node labels:', labels3)

Node labels: ['DataSource', 'MetaNode']


In [126]:
colors3 = random_color_palette(len(labels3))

In [127]:
for label, color in zip(labels3, colors3):
    style3.append({'selector': 'node[label = "' + label + '"]', 'style': {'background-color': color}})

In [128]:
widget3.set_style(style3)

In [129]:
widget3.set_layout(name='klay', padding=0)

In [130]:
widget3.set_tooltip_source('tooltip')

Click on a node to show the tooltip

In [131]:
widget3

CytoscapeWidget(cytoscape_layout={'name': 'klay', 'padding': 0}, cytoscape_style=[{'selector': 'node', 'style'…

### Example 4: Create a Metagraph that shows the Nodes and their Relationships in the COVID-19-Net Knowledge Graph

In [132]:
query4 = """
MATCH p=(a:MetaNode)-[:META_RELATIONSHIP]->(b:MetaNode) 
WHERE a.name <> 'Location' AND b.name <> 'Location' // exclude Location nodes since they make the graph too crowded
RETURN p
"""
subgraph4 = graph.run(query4).to_subgraph()

In [133]:
widget4 = ipycytoscape.CytoscapeWidget()
widget4.graph.add_graph_from_neo4j(subgraph4)

In [134]:
style4 = [node_centered, edge_directed_named]

In [135]:
labels4 = list(subgraph4.labels())
print('Node labels:', labels4)

Node labels: ['MetaNode']


In [136]:
colors4 = random_color_palette(len(labels4))

In [137]:
for label, color in zip(labels4, colors4):
    style4.append({'selector': 'node[label = "' + label + '"]', 'style': {'background-color': color}})

In [138]:
widget4.set_style(style4)

Cola layout [options](https://github.com/cytoscape/cytoscape.js-cola#api)

In [139]:
widget4.set_layout(name='cola', padding=0, nodeSpacing=65, nodeDimensionsIncludeLabels=True, unconstrIter=5000)

In [140]:
widget4.set_tooltip_source('tooltip')

Click on a node to show the node tooltip

In [141]:
widget4

CytoscapeWidget(cytoscape_layout={'name': 'cola', 'padding': 0, 'nodeSpacing': 65, 'nodeDimensionsIncludeLabel…